# 在 Diffusion model -> ESM loss vs. DSM loss


## ESM 與 DSM 理論與目標函數對照

### **1. ESM（Hyvärinen Score Matching）**

直接在原始資料分佈 $p_{\text{data}}(x)$ 上學習 score 函數  
$s_\theta(x) \approx \nabla_x \log p(x)$。

目標函數為：
$$
\mathcal{L}_{\text{ESM}}
= \mathbb{E}_{x}\Big[\tfrac{1}{2}\|s_\theta(x)\|^2 + \operatorname{div}_x s_\theta(x)\Big],
$$

其中散度項可利用 **Hutchinson’s trick** 近似：
$$
\operatorname{tr}(J) \approx v^\top (\partial s / \partial x) v,
$$
$ v $ 為隨機向量（常用 Rademacher 或 Gaussian 噪聲）。

---

#### **2. DSM（Denoising Score Matching）**

在加噪後的平滑分佈  
$$
q_\sigma = p * \mathcal{N}(0, \sigma^2 I)
$$
上學習 denoised score：
$$
s_\theta(y, \sigma) \approx \nabla_y \log q_\sigma(y),
$$
其中 $ y = x + \sigma z, \; z \sim \mathcal{N}(0, I) $。

目標函數為：
$$
\mathcal{L}_{\text{DSM}}
= \mathbb{E}\Big[\lambda(\sigma)
   \big\|s_\theta(y, \sigma) - \tfrac{x - y}{\sigma^2}\big\|^2\Big].
$$

常見選擇是 $\lambda(\sigma) = \sigma^2$ 來平衡不同噪聲層級。

---

### **3. 理論對齊關係**

- DSM 學的是「**平滑後的 score**」$ \nabla_y \log q_\sigma(y) $。  
- 當 $ \sigma \to 0 $ 時，$ q_\sigma \to p $，此時 DSM 與 ESM 等價（相差常數項）。

因此：
- 在固定 $ \sigma > 0$ 時，兩種 loss 不會完全一致。  
- 若令 $ \sigma $ 很小，DSM 模型在 **ESM 評估** 下會趨近於 ESM 模型。  
- 同樣地，ESM 模型在小 $\sigma$ 的 **DSM 目標** 下，其 loss 也會與 DSM 模型接近。

---